In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import csv
import re
import nltk

In [ ]:
file_path = 'train1.xlsx'
df = pd.read_excel(file_path)
df

In [ ]:
def label(a):
  if a == 'Non Negatif' :
    return 0
  else :
    return -1

df['Aktual'] = df['Sentimen'].apply(label)
df

In [ ]:
#CASE FOLDING
df["Case_Folding"] = df["Comment"].str.lower()
df

In [ ]:
#CONVERT EMOJI
emoji_dict = {
    "😊": "bahagia", "😢": "sedih", "🥲": "tersentuh", "😓": "sedih",
    "❤️": "suka", "❤": "suka", "👍": "suka", "🫰": "suka", "💗": "suka",
    "👎": "tidak suka", "🎉": "meriah", "🤔": "berpikir", "😂": "tertawa",
    "🤣": "tertawa", "😍": "terpesona", "🙌": "pujian", "😭": "menangis",
    "🤲": "harapan", "😔": "sedih", "😌": "sedih", "😞": "sedih", "✅": "iya",
    "✔️": "iya", "❌": "tidak", "🔥": "keren", "✨": "dukungan", "✔": "iya",
    "😳": "takjub", "🤩": "takjub", "🥺": "memohon", "😩": "lelah",
    "😁": "antusias", "😎": "keren", "😄": "bahagia", "🤭": "bercanda",
    "🙏": "harapan", "🙏🏻": "harapan", "🫢": "bungkam", "😅": "canggung",
    "🥰": "bahagia", "👏": "apresiasi", "👏🏻": "apresiasi", "👏🏽": "apresiasi",
    "😻": "suka", "🫡": "bangga", "😃": "bahagia", "😲": "takjub",
    "😆": "gembira", "💖": "suka", "💛": "suka", "💙": "suka", "💕": "suka",
    "💓": "suka", "♥️": "suka", "🤍": "suka", "🖤": "suka", "👍🏻": "suka",
    "🤯": "takjub", "💯": "keren", "👍🏽": "suka", "🥹": "bangga",
    "😏": "sindiran", "😒": "kesal", "😫": "kesal", "🤗": "terimakasih",
    "👏🏼": "apresiasi", "👍🏻": "suka", "🫶": "suka"}

# Membuat fungsi untuk mengubah emoji ke teks
def convert_emoji(text):
    for emoji, replacement in emoji_dict.items():
        text = text.replace(emoji, replacement)
    return text

# Mengganti emoji dalam kolom full_text
df['Convert_Emoji'] = df['Case_Folding'].apply(convert_emoji)
df

In [ ]:
#CLEANSING
def cleansing(text):
#menghapus url
  text = re.sub(r'(https?://\S+|www\.\S+)|(www\.[^\s]+)',' ', text)
#menghapus hashtag
  text = re.sub(r'#([^\s]+)',' ', text)
#menghapus username
  text = re.sub(r'@([^\s]+)',' ', text)
#menghapus punctuation
  text = re.sub(r'[!$%^&*@#()_+±|~=`{}\[\]%\-:";\'<>‘’?,.“”\/]',' ', text)
#menghapus digit
  text = re.sub(r'\d+',' ',text)
#menghapus whitespace
  text = re.sub(r'[\s]+',' ', text)
  return text

df['Cleansing'] = df['Convert_Emoji'].apply(cleansing)
df

In [ ]:
#TOKENIZING
from nltk.tokenize import word_tokenize
nltk.download('punkt')

def tokenizing(text):
  tokens = nltk.word_tokenize(text)
  return tokens

df['Tokenizing'] = df['Cleansing'].apply(tokenizing)
df

In [ ]:
file_path1 = 'colloquial-indonesian-lexicon.csv'
normalized_lexicon = pd.read_csv(file_path1)
normalized_lexicon

In [ ]:
file_path2 = 'normalisasi.xlsx'
normal_word = pd.read_csv(file_path2)
normal_word

In [ ]:
normalized_word = pd.concat([normalized_lexicon, normal_word], ignore_index=True)
normalized_word

In [ ]:
#NORMALIZATION
normalized_word_dict = {}

for index, row in normalized_word.iterrows():
    if row[0] not in normalized_word_dict:
        normalized_word_dict[row[0]] = row[1]

def normalized_term(document):
    return [normalized_word_dict[term] if term in normalized_word_dict else term for term in document]

df['Normalization'] = df['Tokenizing'].apply(normalized_term)
df

In [ ]:
#STOPWORD REMOVAL
from nltk.corpus import stopwords
nltk.download('stopwords')

list_stopwords = stopwords.words('indonesian')
list_stopwords

In [ ]:
negation_word = ['belum','belumlah','bukan','bukankah','bukanlah','bukannya','jangan',
                 'jangankan','janganlah','tanpa','tidak','tidakkah','tidaklah']

for word in list(list_stopwords):
  if word in negation_word:
    list_stopwords.remove(word)

In [ ]:
list_stopwords.extend(['hmmm', 'eh', 'kah', 'hi', 'wuih', 'nih', 'nya', 'coy', 'ya', 'gurls', 'dah', 'ah', 'wkwkwk', 'wkwkwkw', 'ni', 'nihhhhhhh', 'nah', 'di', 'si', 'aaaaaa', 'deh', 'cuy', 'gw', 'hehe', 'dsb', 'gue', 'bilamana', 'ber',
                       'kan', 'w', 'wk', 'mb', 'hahahha', 'e', 'pfftttt', 'yuk', 'sih', 'nih', 'lo', 'naah', 'nii', 'jn', 'ini', 'hua', 'ah', 'slurr', 'anjay', 'ny', 'kak', 'min', 'guyss', 'kakkk', 'wkwkwk…', 'sok', 'teng', 'aaaakkk',
                       'mimin','dll', 'plis', 'yaa', 'xd', 'an', 'lu', 'tb', 'yo', 'lohh', 'dehh', 'o', 'r', 'y', 'ehhh', 'wkwkwkwkwk', 'f', 'wkwk', 'guys', 'ke', 'ok', 'siih', 'kokkk', 'ta', 'oh', 'btwww', 'plisss', 'tent', 'duh',
                       'aaah', 'heh', 'sis', 'aaaaa', 'waaa', 'ngl', 'ohh', 'bant', 'wakakaka', 'ama', 'kaa', 'kakk', 'kaka', 'ka', 'lohh', 'ihh', 'hah', 'kaaa', 'niiiiiiiii', 'huhuuu', 'gpp', 'pliss', 'hehe', 'gais', 'srg', 'nge', 'aco',
                       'dllnya', 'wiiihh', 'anjaaay', 'loh', 'woi', 'woy', 'aaaaaaaa', 'huhuhu', 'wkwkwkk', 'huwaaaaa', 'fallawwww', 'fallawwwwwww', 'hai', 'ha', 'haha', 'woooooooo', 'hayooh', 'xixixi', 'yhh', 'lololo', 'ges', 'wkwkwkwk',
                       'nahh', 'xixi', 'wkwkkwk', 'wkkwkwk', 'lho', 'huhu', 'wkwkw', 'btw', 'wkwkwk', 'awokawok', 'hehehe', 'oi', 'awowkowkok', 'huhuhuu', 'ygy', 'yagesya', 'wkkwkw', 'wkwkkww', 'nnya', 'alah', 'cok', 'gass', 'halo',
                       'wkwkwk', 'donh', 'ddos', 'tt', 'bro', 'rw', 'tenggg', 'halah', 'dooonk', 'mah', 'fyp', 'bgst', 'ih', 'huahaha', 'a', 'p', 't', 'h', 'onoo', 'b', 'n', 'dot', 'id'])

list_stopwords = set(list_stopwords)

def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

df['Stopword_Removal'] = df['Normalization'].apply(stopwords_removal)
df

In [ ]:
pip install nlp-id

In [ ]:
#LEMMATIZATION
from nlp_id.lemmatizer import Lemmatizer

lemmatizer = Lemmatizer()

def lemmatize(text):
  if isinstance(text, list):
    lemmatized_tokens = [lemmatizer.lemmatize(str(token)) for token in text]
    return lemmatized_tokens
  else:
    return str(text)

df['Lemmatization'] = df['Stopword_Removal'].apply(lemmatize)
df

In [ ]:
X = df['Lemmatization']
y = df['Aktual']

In [ ]:
from sklearn.model_selection import train_test_split

#SPLIT DATA TESTING DAN DATA TRAINING 90:10
X_train, X_test, y_train, y_test = train_test_split(X.astype('U'), y, test_size=0.1, random_state=42)
print(f'Total training data : {len(X_train)}')
print(f'Total testing data : {len(X_test)}')

In [ ]:
#SPLIT DATA TESTING DAN DATA TRAINING 80:20
X_train2, X_test2, y_train2, y_test2 = train_test_split(X.astype('U'), y, test_size=0.2, random_state=42)
print(f'Total training data : {len(X_train2)}')
print(f'Total testing data : {len(X_test2)}')

In [ ]:
#SPLIT DATA TESTING DAN DATA TRAINING 70:30
X_train3, X_test3, y_train3, y_test3 = train_test_split(X.astype('U'), y, test_size=0.3, random_state=42)
print(f'Total training data : {len(X_train3)}')
print(f'Total testing data : {len(X_test3)}')

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer()
#TF-IDF 90:10
tfidf_train = tf.fit_transform(X_train)
tfidf_test = tf.transform(X_test)
print(tfidf_train,'\n')
print(tfidf_test)

In [ ]:
#TF-IDF 80:20
tfidf_train2 = tf.fit_transform(X_train2)
tfidf_test2 = tf.transform(X_test2)

In [ ]:
#TF-IDF 70:30
tfidf_train3 = tf.fit_transform(X_train3)
tfidf_test3 = tf.transform(X_test3)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
NB_classifier = MultinomialNB()
#TRAINING MODEL MENGGUNAKAN DATA LATIH 90:10
classifier = NB_classifier.fit(tfidf_train, y_train)
#PREDIKSI PADA DATA UJI
y_pred = classifier.predict(tfidf_test)

In [ ]:
#TRAINING MODEL MENGGUNAKAN DATA LATIH 80:20
classifier2 = NB_classifier.fit(tfidf_train2, y_train2)
#PREDIKSI PADA DATA UJI
y_pred2 = classifier.predict(tfidf_test2)

In [ ]:
#TRAINING MODEL MENGGUNAKAN DATA LATIH 70:30
classifier3 = NB_classifier.fit(tfidf_train3, y_train3)
#PREDIKSI PADA DATA UJI
y_pred3 = classifier.predict(tfidf_test3)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

#CONFUSION MATRIX
cnf_matrix1 = confusion_matrix(y_test,y_pred)
cnf_matrix1

In [ ]:
cnf_matrix2 = confusion_matrix(y_test2,y_pred2)
cnf_matrix2

In [ ]:
cnf_matrix3 = confusion_matrix(y_test3,y_pred3)
cnf_matrix3

In [ ]:
#report 90:10
print(classification_report(y_test,y_pred))

In [ ]:
#report 80:20
print(classification_report(y_test2,y_pred2))

In [ ]:
#report 70:30
print(classification_report(y_test3,y_pred3))

In [ ]:
group_names = ['TN','FP','FN','TP']
group_counts = ["{0:0.0f}".format(value) for value in cnf_matrix3.flatten()]
labels = [f"{v1}\n{v2}" for v1, v2 in zip(group_names,group_counts)]
labels = np.asarray(labels).reshape(2,2)
sns.heatmap(cnf_matrix3, annot=labels, fmt='', cmap='Blues');

In [ ]:
X_train_df = pd.DataFrame({'Lemmatization': X_train3, 'Label': y_train3})
X_train_df

In [ ]:
neg_train = X_train_df[X_train_df['Label'] == -1]
neg_train

In [ ]:
X_test_df = pd.DataFrame({'Lemmatization': X_test3, 'Label': y_test3})
X_test_df

In [ ]:
labelling=[]
for i, text in enumerate(X_test3):
  labelling.append({"Lemmatization" : text, "Label": y_pred3[i]})

labelling_output = pd.DataFrame(labelling)
print(labelling_output)

In [ ]:
neg_test = labelling_output[labelling_output['Label'] == -1]
neg_test

In [ ]:
file_path = 'test1.xlsx'
df_test = pd.read_excel(file_path)
df_test

In [ ]:
#CASE FOLDING
df_test["Case_Folding"] = df_test["Comment"].str.lower()
df_test

In [ ]:
#CONVERT EMOJI
emoji_dict = {
    "😊": "bahagia", "😢": "sedih", "🥲": "tersentuh", "😓": "sedih",
    "❤️": "suka", "❤": "suka", "👍": "suka", "🫰": "suka", "💗": "suka",
    "👎": "tidak suka", "🎉": "meriah", "🤔": "berpikir", "😂": "tertawa",
    "🤣": "tertawa", "😍": "terpesona", "🙌": "pujian", "😭": "menangis",
    "🤲": "harapan", "😔": "sedih", "😌": "sedih", "😞": "sedih", "✅": "iya",
    "✔️": "iya", "❌": "tidak", "🔥": "keren", "✨": "dukungan", "✔": "iya",
    "😳": "takjub", "🤩": "takjub", "🥺": "memohon", "😩": "lelah",
    "😁": "antusias", "😎": "keren", "😄": "bahagia", "🤭": "bercanda",
    "🙏": "harapan", "🙏🏻": "harapan", "🫢": "bungkam", "😅": "canggung",
    "🥰": "bahagia", "👏": "apresiasi", "👏🏻": "apresiasi", "👏🏽": "apresiasi",
    "😻": "suka", "🫡": "bangga", "😃": "bahagia", "😲": "takjub",
    "😆": "gembira", "💖": "suka", "💛": "suka", "💙": "suka", "💕": "suka",
    "💓": "suka", "♥️": "suka", "🤍": "suka", "🖤": "suka", "👍🏻": "suka",
    "🤯": "takjub", "💯": "keren", "👍🏽": "suka", "🥹": "bangga",
    "😏": "sindiran", "😒": "kesal", "😫": "kesal", "🤗": "terimakasih",
    "👏🏼": "apresiasi", "👍🏻": "suka", "🫶": "suka"
}

# Membuat fungsi untuk mengubah emoji ke teks
def convert_emoji(text):
    for emoji, replacement in emoji_dict.items():
        text = text.replace(emoji, replacement)
    return text

# Mengganti emoji dalam kolom full_text
df_test['Convert_Emoji'] = df_test['Case_Folding'].apply(convert_emoji)
df_test

In [ ]:
#CLEANSING
def cleansing(text):
#menghapus url
  text = re.sub(r'(https?://\S+|www\.\S+)|(www\.[^\s]+)',' ', text)
#menghapus hashtag
  text = re.sub(r'#([^\s]+)',' ', text)
#menghapus username
  text = re.sub(r'@([^\s]+)',' ', text)
#menghapus punctuation
  text = re.sub(r'[!$%^&*@#()_+±|~=`{}\[\]%\-:";\'<>‘’?,.“”\/]',' ', text)
#menghapus digit
  text = re.sub(r'\d+',' ',text)
#menghapus whitespace
  text = re.sub(r'[\s]+',' ', text)
  return text

df_test['Cleansing'] = df_test['Convert_Emoji'].apply(cleansing)
df_test

In [ ]:
#TOKENIZING
from nltk.tokenize import word_tokenize
nltk.download('punkt')

def tokenizing(text):
  tokens = nltk.word_tokenize(text)
  return tokens

df_test['Tokenizing'] = df_test['Cleansing'].apply(tokenizing)
df_test

In [ ]:
#NORMALIZATION
normalisasi_kata_dict = {}

for index, row in normalized_word.iterrows():
    if row[0] not in normalisasi_kata_dict:
        normalisasi_kata_dict[row[0]] = row[1]

def normalisasi_kata(document):
    return [normalisasi_kata_dict[term] if term in normalisasi_kata_dict else term for term in document]

df_test['Normalization'] = df_test['Tokenizing'].apply(normalisasi_kata)
df_test

In [ ]:
#STOPWORD REMOVAL
df_test['Stopword_Removal'] = df_test['Normalization'].apply(stopwords_removal)
df_test

In [ ]:
#LEMMATIZATION
df_test['Lemmatization'] = df_test['Stopword_Removal'].apply(lemmatize)
df_test

In [ ]:
dfTest_selected = df_test[['Id', 'Comment', 'Stopword_Removal']]

In [ ]:
text_combined = [' '.join(tokens) for tokens in df_test['Lemmatization']]
tfidf_newtest = tf.transform(text_combined)
y_pred_new = classifier3.predict(tfidf_newtest)
label=[]
for i, text in enumerate(df_test['Lemmatization']):
    label.append({"Lemmatization": text, "Label": y_pred_new[i]})

output = pd.DataFrame(label)
print(output)

In [ ]:
df_combined = pd.concat([dfTest_selected, output], axis=1)
df_combined